In [1]:
import pandas as pd
import numpy as np

In [2]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

# for read-only access, use this line
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('data/credentials.json', scope)
gc = gspread.authorize(credentials)

In [3]:
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1TmN1nD42C3JraxD8Xj-AQ7qeYPVJrDoERArljj-F_co/edit#gid=0")
worksheet = sheet.get_worksheet(0) # sheet index in spreadsheets

In [4]:
from gspread_dataframe import set_with_dataframe
df = pd.DataFrame(worksheet.get_all_records())
#set_with_dataframe(worksheet, df)
df

,รุ่น,ระดับความชอบในการเรียนคณะไอทีลาดกระบัง,ชั้นปี,สาขา,แขนง,รอบการสมัคร,เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง,มีพื้นฐานเกี่ยวกับคอมพิวเตอร์หรือการเขียนโปรแกรมมาก่อนไหม ภาษาอะไรบ้าง (ไม่ว่าจะเป็นการตัดต่อ/เขียนโปรแกรม/ออกแบบ),ตั้งใจจะเข้าไอทีลาดกระบังตั้งแต่แรกไหม,ถ้าไม่ได้ตั้งใจจะเข้าไอทีลาดกระบังตั้งแต่แรก มีเล็งคณะ/มหาลัยอื่นไว้อีกไหม อะไรบ้าง,...,สายที่เรียนตอนม.ปลาย,ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาคณิตศาสตร์],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาสังคม],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาวิทยาศาสตร์],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาศิลปะ],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาเทคโนโลยี],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาพลศึกษา],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาภาษาต่างประเทศ],ถ้าเคยคิดจะซิ่ว จะซิ่วเพราะอะไร,target
0,60,1,ปี 4,IT,Multimedia and Game Development,Admission 60,หางานง่ายดี อาชีพที่มั่นคง และใกล้บ้าน,กราฟฟิค,ไม่ใช่,บางมด (ไอที),...,วิทย์-คณิต,เฉยๆ,ไม่ชอบ,เฉยๆ,ชอบ,เฉยๆ,ไม่ชอบ,เฉยๆ,เพราะไม่ใช่ทาง เรียนแล้วไม่ฉลาดขึ้น,ไม่เหมาะ
1,60,2,ปี 4,DSBA,-,Admission 60,เพราะมาopen houseแล้วรู้สึกถูกชะตากับที่นี่ สถ...,"การตัดต่อภาพ, adobe dreamweaver",ไม่ใช่,วิศวะอาหาร ลาดกระบัง,...,วิทย์-คณิต,เฉยๆ,เฉยๆ,ไม่ชอบ,ไม่ชอบ,ชอบ,เฉยๆ,ไม่ชอบ,เกรดไม่ดี คิดว่าไม่น่าไหว,ไม่เหมาะ
2,60,2,ปี 4,IT,Software Engineering,สอบตรง 60,ชอบเป็นลำดับรองลงมาเนื่องจากพ่อแม่ไม่ให้เรียนน...,"C, C++, Java, Web (CSS/HTML), การตัดต่อภาพ",ไม่ใช่,นิติศาสตร์,...,วิทย์-คอม,เฉยๆ,ชอบ,ชอบ,ไม่ชอบ,ชอบ,ชอบ,ชอบ,-,ไม่เหมาะ
3,61,3,ปี 3,IT,Software Engineering,TCAS2 ปี61,สนใจ,"C, กราฟฟิค",ใช่,-,...,วิทย์-คณิต,เฉยๆ,เฉยๆ,เฉยๆ,เฉยๆ,เฉยๆ,เฉยๆ,เฉยๆ,-,ไม่เหมาะ
4,60,3,ปี 4,IT,Network and System Technology,สอบตรง 60,เพราะในช่วงที่เรียนมัธยมนั้นไม่ได้มีความรู้สึก...,ไม่มีพื้นฐานอะไรเลย,ใช่,-,...,วิทย์-คณิต,เฉยๆ,เฉยๆ,เฉยๆ,ชอบ,เฉยๆ,เฉยๆ,ไม่ชอบ,คิดว่าตัวเองไม่ไหวแต่พอคิดต่อว่าถ้าซิ่วแล้วจะเ...,ไม่เหมาะ
5,61,3,ปี 3,IT,Software Engineering,TCAS2 ปี61,ตอนมอหก มาสอบตรงที่นี่แล้วติด เลยเอาเลย สนใจด้...,"การตัดต่อวีดีโอ, การตัดต่อภาพ",ไม่ใช่,บัญชี พระนครเหนือ บางมด,...,วิทย์-คณิต,ไม่ชอบ,เฉยๆ,เฉยๆ,ชอบ,ชอบ,เฉยๆ,ชอบ,เพราะว่าเรื่องเกรด ปีหนึ่งยังรู้สึกยาก แล้วเพื...,ไม่เหมาะ
6,60,3,ปี 4,DSBA,-,สอบตรง 60,สนใจคอม และติดที่แรก เลยเอาเลย,"การตัดต่อวีดีโอ, Hardware",ใช่,-,...,วิทย์-คณิต,ไม่ชอบ,ไม่ชอบ,ชอบ,ไม่ชอบ,ชอบ,ชอบ,ไม่ชอบ,-,ไม่เหมาะ
7,60,3,ปี 4,IT,Software Engineering,สอบตรง 60,เนื่องด้วยส่วนตัวเป็นคนชอบเทคโนโลยีอะไรใหม่ๆ ไ...,"C, C#, Java, Web (CSS/HTML), การตัดต่อวีดีโอ, ...",ใช่,รัฐศาสตร์ (เยอรมัน) จุฬา,...,วิทย์-คณิต,เฉยๆ,เฉยๆ,เฉยๆ,ชอบ,ชอบ,ไม่ชอบ,ชอบ,เพราะไอทีเป็นความถนัดลำดับที่ 2 รองลงมาจากด้าน...,ไม่เหมาะ
8,61,3,ปี 3,IT,Network and System Technology,TCAS2 ปี61,คิดว่าที่เลือกเรียนคณะเทคโนโลยีสาระสนเทศเพราะว...,"การตัดต่อภาพ, กราฟฟิค",ไม่ใช่,คณะสถาปัตยกรรม ที่ ลาดกระบัง,...,วิทย์-คณิต,ชอบ,เฉยๆ,ชอบ,ชอบ,เฉยๆ,ชอบ,ไม่ชอบ,เคยคิดจะซิ่วตอนปี 2 เทอม 1 เพราะไม่เข้าใจวิชา ...,ไม่เหมาะ
9,60,3,ปี 4,DSBA,-,รับตรง (ยื่นคะแนน),สนใจและอยากศึกษาในสายงานที่เกี่ยวกับ data anal...,ไม่มีพื้นฐานอะไรเลย,ใช่,-,...,ศิลป์-คำนวณ,เฉยๆ,ชอบ,เฉยๆ,ไม่ชอบ,เฉยๆ,ชอบ,เฉยๆ,-,ไม่เหมาะ


In [5]:
#df = pd.read_excel("data.xlsx")

df = df.drop([38])
df = df.drop(columns=['รุ่น','ระดับความชอบในการเรียนคณะไอทีลาดกระบัง','ชั้นปี','สาขา',
                        'แขนง','รอบการสมัคร','ถ้าเคยคิดจะซิ่ว จะซิ่วเพราะอะไร',
                        'หลังจากเข้ามาเรียนแล้วความรู้สึกเปลี่ยนไปจากตอนแรกไหม เพราะอะไร',
                       'เคยคิดจะซิ่วไหม'])

In [6]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split



def get_word(text):
    tokens = word_tokenize(text,engine='newmm',keep_whitespace=False)
    return tokens


def tokenize(df):
    lst_token = []
    for t in range(len(df)):
        word = get_word(df[t])
        lst_token.append(word)
    return lst_token
lst_token = tokenize(df['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง'])


def stopwords(lst_token):
    stopwords = list(thai_stopwords())
    stopwords.remove('ไม่')
    remove_words = ['(',')','1.','2.','3.','4.','5','6','\u200b','“','”']
    for i in range(len(remove_words)):
        stopwords.append(remove_words[i])

    lst_not_in_stopwords = []
    for i in range(len(lst_token)):
        list_word_not_stopwords = [j for j in lst_token[i] if j not in stopwords]
        lst_not_in_stopwords.append(list_word_not_stopwords)
        
    lst_stw = []
    for j in range(len(lst_not_in_stopwords)):
        list_no_stopwords = [k for k in lst_not_in_stopwords[j] if k != " "]
        lst_stw.append(list_no_stopwords)
    
    reasons_lst = []
    for i in range(len(lst_stw)): #stopwords(lst_token) = lst_stw
        reasons_lst.append(''.join(lst_stw[i]))
    
    return reasons_lst
reasons_lst = stopwords(lst_token)



In [7]:
reasons_lst

['หางานดีอาชีพมั่นคงบ้าน',
 'openhouseรู้สึกถูกชะตาที่นี่สถานที่รุ่นพี่',
 'ชอบเป็นลำดับรองลงมาพ่อแม่ไม่เรียนนิติศาสตร์',
 'สนใจ',
 'เรียนมัธยมไม่มีความรู้สึกชอบวิชาคืดเล่นคอมทำงานคอมไอที',
 'ตอนมอหกสอบที่นี่ติดเอาเลยสนใจเทคโนโลยีไม่สนใจโค้ดดิ้ง',
 'สนใจคอมติดเอาเลย',
 'เนื่องด้วยส่วนตัวคนชอบเทคโนโลยีศึกษาทดลองทำเรียนต่อที่นี่ความชอบกลายเป็นทำได้งานอดิเรกกลายเป็นอาชีพตัดสินใจเข้าเรียนที่นี่',
 'ว่าที่เลือกเรียนคณะเทคโนโลยีสาระสนเทศตอนนี้ระบบAIตอนนี้บริษัทปลดพนักงานจำนวนมากระบบAIแทนAIการเรียนผมเข้าเรียนจบงานเป็นที่ต้องการบริษัทจำนวนมากผมรู้ตัวเองตอนม.ลองสอบดู',
 'สนใจศึกษาสายงานdataanalyticsคณะเทคโนโลยีสารสนเทศสูตรเลือกที่นี่',
 'ดูน่าสนใจสนใจคอม',
 'สาขาน่าสนใจสนใจมัลติเดินทางสะดวก',
 'คณะไอทีลาดกระบังคณะชั้นนำมีชื่อเสียงสถานที่เรียนน่าอยู่',
 'ตอนแรกเรียนคอมไม่เก่งวิทย์หาข้อมูลที่นี่ไม่วิทย์',
 'เพื่อนชวนToBeITติดสอบTopสนใจคณะ',
 'สนใจคอมอยู่แล้วคณะรองรับโควต้ากิจกรรม',
 'ดาต้าดาต้าสนใจดาต้ายูทูปฟีเจอร์recommendแพชชั่นศึกษาศาสตร์',
 'คณะที่อยู่ช้อยส์ที่ตั้งคณะไม่เจอวิชาวิทยาศาสตร์ไม่ต

In [8]:
#TFIDF
tvec = TfidfVectorizer(tokenizer=get_word, ngram_range=(1,3))
tvec.fit(reasons_lst)
reason_vec = tvec.transform(reasons_lst).todense()
feature_names = tvec.get_feature_names()
corpus_index = [n for n in df['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
df_vec_reason = pd.DataFrame(reason_vec, index=corpus_index, columns=feature_names)

C:\Users\User\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [9]:
reason_vec

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.12441934, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [10]:
#EXP


def exp(dataframe):
    df_exp = pd.DataFrame(dataframe['มีพื้นฐานเกี่ยวกับคอมพิวเตอร์หรือการเขียนโปรแกรมมาก่อนไหม ภาษาอะไรบ้าง (ไม่ว่าจะเป็นการตัดต่อ/เขียนโปรแกรม/ออกแบบ)'])
    df_exp = dataframe.rename(columns={'มีพื้นฐานเกี่ยวกับคอมพิวเตอร์หรือการเขียนโปรแกรมมาก่อนไหม ภาษาอะไรบ้าง (ไม่ว่าจะเป็นการตัดต่อ/เขียนโปรแกรม/ออกแบบ)':'exp'})
    df_exp['new'] = df_exp['exp'].str.split(', ')

    lst_all_exp = ['Programming','Python','Web Programming','App Programing','Graphic','No exp']
    df_vec_exp = pd.DataFrame(columns=lst_all_exp)


    lst_exp_1 = ['C','C++',"C#","Java"]
    lst_exp_2 = ["Python"]
    lst_exp_3 = ["Web (CSS/HTML)", "JavaScript", "PHP"]
    lst_exp_4 = ["dart"]
    lst_exp_5 = ["adobe dreamweaver", "กราฟฟิค", "การตัดต่อภาพ", "การตัดต่อวีดีโอ"]
    lst_exp_6 = ["ไม่มีพื้นฐานอะไรเลย"]

    for i in range(len(df_exp)):
        value = df_exp['new'].iloc[i]

        for j in range(len(value)):
            words = value[j]
            if words in lst_exp_1 :
                df_vec_exp.loc[i,'Programming'] = 1
            if words in lst_exp_2 :
                df_vec_exp.loc[i,'Python'] = 1
            if words in lst_exp_3 :
                df_vec_exp.loc[i,'Web Programming'] = 1
            if words in lst_exp_4 :
                df_vec_exp.loc[i,'App Programing'] = 1
            if words in lst_exp_5 :
                df_vec_exp.loc[i,'Graphic'] = 1
            if words in lst_exp_6 :
                df_vec_exp.loc[i,'No exp'] = 1

        #lst_exp = list(OrderedDict.fromkeys(lst_exp))
        #print(lst_exp)
        #lst_all.append(lst_exp)
    df_exp = df_vec_exp.fillna(0)
    
    corpus_index = [n for n in dataframe['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    for i in range(len(dataframe)):
        df_exp = df_exp.rename(index={i: corpus_index[i]})
    
    return df_exp
    
dummies_exp = exp(df)

In [11]:
# ตั้งใจจะเข้าไอทีลาดกระบังตั้งแต่แรกหรือไม่
def entrance(dataframe):
    df_entrance_data = dataframe['ตั้งใจจะเข้าไอทีลาดกระบังตั้งแต่แรกไหม']
    lst_entrance = []
    for i in range(len(dataframe)):
        if df_entrance_data[i] == 'ใช่':
            lst_entrance.append(1)
        else:
            lst_entrance.append(0)


    df_entrance = pd.DataFrame(lst_entrance)

    #To Dataframe
    corpus_index = [n for n in dataframe['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    for i in range(len(dataframe)):
        df_entrance = df_entrance.rename(index={i: corpus_index[i]})
        #print(dummies_entrance)
    df_entrance = df_entrance.rename(columns={0: 'ตั้งใจจะเข้าไอทีลาดกระบังตั้งแต่แรกไหม'})
    
    return df_entrance

dummies_entrance = entrance(df)

In [12]:
#สายที่เรียนตอนม.ปลาย
def major(dataframe):
    df_major_data = dataframe['สายที่เรียนตอนม.ปลาย']
    df_major = pd.get_dummies(df_major_data.apply(pd.Series).stack()).sum(level=0)
    
    corpus_index = [n for n in dataframe['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    for i in range(len(dataframe)):
        df_major = df_major.rename(index={i: corpus_index[i]})
    
    return df_major

dummies_major = major(df)

In [13]:
#ระดับความชอบของกลุ่มวิชา
def subjects(dataframe):
    df_subjects_data = dataframe[['ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาคณิตศาสตร์]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาสังคม]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาวิทยาศาสตร์]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาศิลปะ]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาเทคโนโลยี]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาพลศึกษา]',
                                  'ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาภาษาต่างประเทศ]']]
    df_subjects_data = df_subjects_data.replace(["เฉยๆ"],"2")
    df_subjects_data = df_subjects_data.replace(["ไม่ชอบ"],"1")
    df_subjects_data = df_subjects_data.replace(["ชอบ"],"3")
    df_subjects = df_subjects_data
    
    corpus_index = [n for n in dataframe['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    for i in range(len(dataframe)):
        df_subjects = df_subjects.rename(index={i: corpus_index[i]})

    return df_subjects

dummies_subjects = subjects(df)
    

In [14]:

#สำหรับtrain data
df_target = df['target']
corpus_index = [n for n in df['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
for i in range(len(df)):    
    df_target = df_target.rename(index={i: corpus_index[i]})
df_target = pd.DataFrame(df_target)

df_vector = pd.concat([df_vec_reason, dummies_exp, dummies_entrance, dummies_major, dummies_subjects, df_target], axis=1)



#MERGE สำหรับรับinput
def merge(df_reason,df_exp,df_entrance,df_major,df_subject):
    df_allvector = pd.concat([df_reason,df_exp,df_entrance,df_major,df_subject], axis=1)
    return df_allvector

In [15]:
df_vector

,ai,ai การเรียน,ai การเรียน ผม,ai ตอนนี้,ai ตอนนี้ บริษัท,ai แทน,ai แทน ai,bi,dataanalytics,dataanalytics คณะ,...,ศิลป์-คำนวณ,ศิลป์-ภาษา,ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาคณิตศาสตร์],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาสังคม],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาวิทยาศาสตร์],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาศิลปะ],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาเทคโนโลยี],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาพลศึกษา],ระดับความชอบของกลุ่มวิชาต่อไปนี้ [กลุ่มวิชาภาษาต่างประเทศ],target
หางานง่ายดี อาชีพที่มั่นคง และใกล้บ้าน,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,1,2,3,2,1,2,ไม่เหมาะ
เพราะมาopen houseแล้วรู้สึกถูกชะตากับที่นี่ สถานที่ รุ่นพี่,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,2,1,1,3,2,1,ไม่เหมาะ
ชอบเป็นลำดับรองลงมาเนื่องจากพ่อแม่ไม่ให้เรียนนิติศาสตร์,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,3,3,1,3,3,3,ไม่เหมาะ
สนใจ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,2,2,2,2,2,2,ไม่เหมาะ
เพราะในช่วงที่เรียนมัธยมนั้นไม่ได้มีความรู้สึกชอบวิชาไหนเลย แต่ก็คืดว่าเราก็เล่นคอมหรือทำงานกับคอมบ่อยก็เลยคิดว่ามาเข้าไอทีก็ดี,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,2,2,3,2,2,1,ไม่เหมาะ
ตอนมอหก มาสอบตรงที่นี่แล้วติด เลยเอาเลย สนใจด้านเทคโนโลยี แต่ไม่ได้สนใจโค้ดดิ้งมาก,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,1,2,2,3,3,2,3,ไม่เหมาะ
สนใจคอม และติดที่แรก เลยเอาเลย,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,1,1,3,1,3,3,1,ไม่เหมาะ
เนื่องด้วยส่วนตัวเป็นคนชอบเทคโนโลยีอะไรใหม่ๆ ได้ศึกษาและทดลองทำด้วยตนเองมาบ้างจึงคิดว่าหากได้เข้าเรียนต่อที่นี่บางทีจากความชอบอาจกลายเป็นอะไรที่เราสามารถทำได้จริงๆ มากกว่าการเป็นงานอดิเรกอาจกลายเป็นอาชีพก็ได้ เลยตัดสินใจเข้าเรียนที่นี่,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,2,2,2,3,3,1,3,ไม่เหมาะ
คิดว่าที่เลือกเรียนคณะเทคโนโลยีสาระสนเทศเพราะว่าในตอนนี้เกือบทุกอย่างใช้ระบบ AI ซึ่งตอนนี้บางบริษัทก็ปลดพนักงานไปจำนวนมากแล้วมาใช้ระบบ AI แทน ซึ่งจะมี AI ได้ก็คงต้องพึ่งการเรียนในด้านนี้ ผมเลยคิดว่า ถ้าได้เข้าเรียนแล้วจบมา งานมันค่อนข้างกว้างและน่าจะเป็นที่ต้องการของบริษัทจำนวนมาก ผมก็พึ่งมารู้ตัวเองตอน ม.6 นี่แหละเลยคิดว่าเราน่าจะลองพยายามสอบดู,0.283842,0.094614,0.094614,0.094614,0.094614,0.094614,0.094614,0.000000,0.000000,0.000000,...,0,0,3,2,3,3,2,3,1,ไม่เหมาะ
สนใจและอยากศึกษาในสายงานที่เกี่ยวกับ data analytics ซึ่งคณะเทคโนโลยีสารสนเทศมีหลังสูตรที่เกี่ยวข้อง จึงเลือกที่นี่,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222134,0.222134,...,1,0,2,3,2,1,2,3,2,ไม่เหมาะ


Model สำหรับรวมกันเป็นเวคเตอร์เดียว

In [16]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = df_vector.drop(columns = ['target'])
y = df_vector['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf_naive = MultinomialNB(alpha = 0.1).fit(X_train, y_train)
naive_pred = clf_naive.predict(X_test)
#print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, naive_pred))


#SVM
from sklearn.svm import SVC
clf_svm = SVC(C=1.0, decision_function_shape = 'ovr', gamma =  1, random_state = 10, kernel = 'linear')
clf_svm.fit(X_train, y_train)
svm_pred = clf_svm.predict(X_test)
#accuracy = sum(y_test==svm_pred)/len(y_test)
#print('   ', accuracy)

#KNN
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors = 8)
clf_knn.fit(X_train, y_train)
knn_pred = clf_knn.predict(X_test)
#print("KNN:",metrics.accuracy_score(y_test, knn_pred))

#Logistic regression
from sklearn.linear_model import LogisticRegression
clf_logis = LogisticRegression(penalty = 'l2',solver = 'liblinear', multi_class = 'ovr', C = 1,intercept_scaling = 0.01)
clf_logis.fit(X_train, y_train)
logis_pred = clf_logis.predict(X_test)
#print("Logistig Regression:",metrics.accuracy_score(y_test, logis_pred))

#Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train,y_train)
decisiontree_pred = clf_tree.predict(X_test)
#print("Accuracy:",metrics.accuracy_score(y_test, decisiontree_pred))

#RandomForest
from sklearn.ensemble import RandomForestClassifier
clf_randomforest = RandomForestClassifier(n_estimators=1000, random_state=0)
clf_randomforest.fit(X_train, y_train)
randomforest_pred = clf_randomforest.predict(X_test)
#print("RandomForest Accuracy:",metrics.accuracy_score(y_test, randomforest_pred))



#ENSEMBLE
from sklearn.ensemble import VotingClassifier

estimators = []
estimators.append(('Naive Bayes', clf_naive))
estimators.append(('SVM', clf_svm))
estimators.append(('RandomForest', clf_randomforest))
estimators.append(('KNN', clf_knn))
estimators.append(('logistic', clf_logis))
estimators.append(('Decision Tree', clf_tree))

from sklearn import model_selection
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train,y_train)
#results = ensemble.score(X_test,y_test) #ไปดูว่าต่างกับข้างล่างยังไง

#results = model_selection.cross_val_score(ensemble, X, y)


#print(results.mean())






VotingClassifier(estimators=[('Naive Bayes',
                              MultinomialNB(alpha=0.1, class_prior=None,
                                            fit_prior=True)),
                             ('SVM',
                              SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma=1, kernel='linear', max_iter=-1,
                                  probability=False, random_state=10,
                                  shrinking=True, tol=0.001, verbose=False)),
                             ('RandomForest',
                              Random...
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                   

In [38]:
results

array([0.5       , 0.75      , 0.5       , 0.85714286, 0.85714286])

Model สำหรับแยกเวคเตอร์แล้วเทรนทีละหัวข้อ สุดท้ายเอามารวมกันโดยให้วิธีโหวตของensemble

In [ ]:
#พวกโมเดลตรงนี้เอาไว้ท้ายสุด ต้องรวมทุกอย่างเป็นเวคเตอร์เดียวกันก่อน


X = tvec.transform(reasons_lst).todense() #ผลลัพธ์เหมือนกับtodense แต่ todense ออกมาเป็นmatrix ส่วนtoarrayออกมาเป็นarray
y = df['target']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.preprocessing import normalize

clf_svm = SVC(C=1.0, decision_function_shape = 'ovr', gamma =  1, random_state = 10, kernel = 'linear')
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)
#accuracy = sum(y_test==y_pred)/len(y_test)

In [24]:

def getintent(dataframe):
    keyword_reason = dataframe['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']
    tokenize_word = tokenize(keyword_reason)
    remove_stopwords = stopwords(tokenize_word)
    tfidf_vec = tvec.transform(remove_stopwords).toarray()
    feature_names = tvec.get_feature_names()
    corpus_index = keyword_reason #ตออนแรกคือ [n for n in df['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    df_reason = pd.DataFrame(tfidf_vec, index=corpus_index, columns=feature_names)
    
    #keyword_exp = pd.DataFrame(dataframe['มีพื้นฐานเกี่ยวกับคอมพิวเตอร์หรือการเขียนโปรแกรมมาก่อนไหม ภาษาอะไรบ้าง (ไม่ว่าจะเป็นการตัดต่อ/เขียนโปรแกรม/ออกแบบ)'])
    df_exp = exp(dataframe)
    
    df_entrance = entrance(dataframe)
    
    
    #https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data
    df_major = major(dataframe)
    # Get missing columns in the training test
    missing_cols = set( dummies_major.columns ) - set( df_major.columns )
    # Add a missing column in test set with default value equal to 0
    for c in missing_cols:
        df_major[c] = 0
    # Ensure the order of column in the test set is in the same order than in train set
    df_major = df_major[dummies_major.columns]
    
    
    df_subjects = subjects(dataframe)
    
    
    df_allvector = merge(df_reason,df_exp,df_entrance,df_major,df_subjects)
    
    
    naive_pred = clf_naive.predict(df_allvector)
    randomforest_pred = clf_randomforest.predict(df_allvector)
    decisiontree_pred = clf_tree.predict(df_allvector)
    logis_pred = clf_logis.predict(df_allvector)
    knn_pred = clf_knn.predict(df_allvector)
    svm_pred = clf_svm.predict(df_allvector)
    
    estimators = []
    estimators.append(('Naive Bayes', clf_naive))
    estimators.append(('SVM', clf_svm))
    estimators.append(('RandomForest', clf_randomforest))
    estimators.append(('KNN', clf_knn))
    estimators.append(('logistic', clf_logis))
    estimators.append(('Decision Tree', clf_tree))
    
    estimators2 = []
    estimators2.append(('Naive Bayes', naive_pred))
    estimators2.append(('SVM', svm_pred))
    estimators2.append(('RandomForest', randomforest_pred))
    estimators2.append(('KNN', knn_pred))
    estimators2.append(('logistic', logis_pred))
    estimators2.append(('Decision Tree', decisiontree_pred))
    
    #ensemble = VotingClassifier(estimators)
    #ensemble.fit(X_train,y_train)
    #target = ensemble.predict(df_allvector)[0]
    #pred = clf_svm.predict(tfidf_vec)[0]
    #return pred
    return estimators2
    #return target
df_test = pd.read_excel('data/test_data.xlsx')
df_test = pd.DataFrame(df_test.iloc[[-1]])    
getintent(df_test)

#อย่าลืมเช็คตอนเอาไปเปรียบเทียบ

[('Naive Bayes', array(['เหมาะ'], dtype='<U8')),
 ('SVM', array(['เหมาะ'], dtype=object)),
 ('RandomForest', array(['เหมาะ'], dtype=object)),
 ('KNN', array(['ไม่เหมาะ'], dtype=object)),
 ('logistic', array(['เหมาะ'], dtype=object)),
 ('Decision Tree', array(['เหมาะ'], dtype=object))]

In [ ]:
#getintent('ชอบสภาพแวดล้อม และประทับใจรุ่นพี่ตอนมาโอเพ่นเฮ้าส์มากๆค่ะ ถึงตอนนี้ก็ยังรู้สึกดีกับสภาพแวดล้อมนี้อยู่')#.to_excel('data/test_result/test_intent.xlsx')

#ชอบสภาพแวดล้อม และประทับใจรุ่นพี่ตอนมาโอเพ่นเฮ้าส์มากๆค่ะ ถึงตอนนี้ก็ยังรู้สึกดีกับสภาพแวดล้อมนี้อยู่
#อยากเรียนรู้ประสบการณ์เกี่ยวกับการทำเกม และเห็นว่าคณะนี้มีการสอนการทำเกมในแขนงมัลติมีเดีย และอยู่ในมหาลัยที่มีชื่อเสียงของประเทศ

In [ ]:
def getintent(keyword):
    df_test = pd.read_excel('data/test_data.xlsx')
    df_test = pd.DataFrame(df_test.iloc[[-1]])
    keyword = [keyword]
    remove_stopwords = stopwords(keyword)
    tfidf_vec = tvec.transform(remove_stopwords).toarray()
    feature_names = tvec.get_feature_names()
    corpus_index = keyword #ตอนแรกคือ [n for n in df['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง']]
    df_reason = pd.DataFrame(tfidf_vec, index=corpus_index, columns=feature_names)
    
    
    
    
    
    
    
    
    
    
    #pred = clf_svm.predict(tfidf_vec)[0]
    #return pred
    return df_reason

In [ ]:
dd = df_test['เหตุผลที่เลือกเรียนคณะไอทีลาดกระบัง'].tolist
dd

In [ ]:
def getintent(keyword):
  keyword = [keyword]
  feat = vectorizer.transform(keyword)
  feat_norm = normalize(feat)
  pred = model.predict(feat_norm)[0]
  return pred